# Multi-Agent System Demo

This notebook demonstrates how to set up and use a multi-agent system using Google's Agent Development Kit (ADK). The system consists of specialized sub-agents (Billing and Support) coordinated by a main agent.

## Import Required Libraries

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.genai import types
import google.generativeai as genai
from google.adk.models.lite_llm import LiteLlm
import os
import logging

## Configure Logging

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger("<<!!!!!!!>>")

## Configure API Keys

**Note:** In a production environment, you should use environment variables or a secure configuration method rather than hardcoding API keys.

In [ ]:
genai.configure(api_key="")
os.environ['OPENAI_API_KEY'] = ''
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
MODEL_GPT_4O = "gpt-3.5-turbo-0125"

## Define Specialized Sub-Agents

We'll create two specialized agents:
1. Billing Agent - Handles billing and payment-related inquiries
2. Support Agent - Provides technical support and troubleshooting assistance

In [ ]:
# Define specialized sub-agents
billing_agent = LlmAgent(
    name="Billing",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You handle billing and payment-related inquiries.",
    description="Handles billing inquiries."
)

support_agent = LlmAgent(
    name="Support",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="You provide technical support and troubleshooting assistance.",
    description="Handles technical support requests."
)

## Define the Coordinator Agent

The coordinator agent routes user requests to the appropriate specialized agent.

In [ ]:
# Define the coordinator agent
coordinator = LlmAgent(
    name="HelpDeskCoordinator",
    model=LiteLlm(model=MODEL_GPT_4O),
    instruction="Route user requests: Use Billing agent for payment issues, Support agent for technical problems.",
    description="Main help desk router.",
    sub_agents=[billing_agent, support_agent]
)

# For ADK compatibility, the root agent must be named `root_agent`
root_agent = coordinator

## Set Up the Runner

The Runner is responsible for managing the agent execution, sessions, and artifacts.

In [ ]:
runner = Runner(
        app_name="test_agent",
        agent=root_agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),)

## Run a Simulation with a User Query

Let's test our multi-agent system with a sample user query. You can uncomment the query you want to test.

In [ ]:
# Simulate a user query
user_query = "I can't log in gmail, help to give advice."
# user_query = "my billing is not working, help to give advice."
user_id = "test_user"
session_id = "test_session"

# Create a session
session = runner.session_service.create_session(
    app_name="test_agent",
    user_id=user_id,
    state={},
    session_id=session_id,
)

In [ ]:
# Create a content object with the user query
content = types.Content(
    role="user", 
    parts=[types.Part.from_text(text=user_query)]
)

# Run the agent with the correct parameters
events = list(runner.run(
    user_id=user_id, 
    session_id=session.id, 
    new_message=content
))

In [ ]:
# Process the events to get the response
response = ""
if events and events[-1].content and events[-1].content.parts:
    for event in events:
        logger.info(f"Event: {event.author}, Actions: {event.actions}")
        response = "\n".join([p.text for p in events[-1].content.parts if p.text])

print(response)

## Try Different Queries

You can experiment with different user queries to see how the multi-agent system routes and handles them. For example:
- Technical support queries should be routed to the Support agent
- Billing and payment queries should be routed to the Billing agent

In [ ]:
# Function to test different queries
def test_query(query):
    # Create a new session for each test
    test_session_id = f"test_session_{hash(query)}"
    session = runner.session_service.create_session(
        app_name="test_agent",
        user_id=user_id,
        state={},
        session_id=test_session_id,
    )
    
    # Create content and run the agent
    content = types.Content(role="user", parts=[types.Part.from_text(text=query)])
    events = list(runner.run(user_id=user_id, session_id=session.id, new_message=content))
    
    # Process and return the response
    if events and events[-1].content and events[-1].content.parts:
        for event in events:
            logger.info(f"Event: {event.author}, Actions: {event.actions}")
        return "\n".join([p.text for p in events[-1].content.parts if p.text])
    return "No response"

In [ ]:
# Test with a technical support query
tech_query = "I can't access my email account"
print(f"Query: {tech_query}")
print(f"Response: {test_query(tech_query)}")

In [ ]:
# Test with a billing query
billing_query = "I was charged twice for my subscription"
print(f"Query: {billing_query}")
print(f"Response: {test_query(billing_query)}")